In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Attention
from tensorflow.keras.layers import TimeDistributed

In [4]:
NUM_CLASSES = 25
NUM_FRAMES = 100
INPUT_DIM = 2
DATA_DIR = "Data"

In [6]:
def load_data(data_dir):
    X, y = [], []
    for class_label in range(1, NUM_CLASSES + 1):
        class_dir = os.path.join(data_dir, str(class_label))
        if not os.path.exists(class_dir):
            continue
        for csv_file in os.listdir(class_dir):
            file_path = os.path.join(class_dir, csv_file)
            data = pd.read_csv(file_path)
            if len(data) == NUM_FRAMES:  # Ensure the correct number of frames
                X.append(data[['x', 'y']].values)
                y.append(class_label - 1)  # 0-based index for classes
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)
    return X, y

In [7]:
X,y = load_data(DATA_DIR)
y = to_categorical(y, NUM_CLASSES)

In [10]:
sum = 0
name = os.listdir(DATA_DIR)
for i in name:
    a = len(os.listdir(os.path.join(DATA_DIR,i)))
    sum+=int(a)
sum

1783

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [14]:
!nvcc --version  
!nvidia-smi       # Check GPU and driver information

/bin/bash: line 1: nvcc: command not found
Sun Dec 29 03:04:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              N/A / ERR! |   3540MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+------------------------

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, LSTM, Dropout, BatchNormalization

def create_gesture_model(input_shape, num_classes):
    model = Sequential()

    # 1D Convolutional Layers
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))

    # LSTM Layer (no Flatten here)
    model.add(LSTM(128, return_sequences=False, activation='tanh'))
    model.add(Dropout(0.4))

    # Fully Connected Layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',  # Use 'categorical_crossentropy' if labels are one-hot encoded
        metrics=['accuracy']
    )

    return model

# Input shape: (100 frames, 2 features for X and Y)
input_shape = (100, 2)
num_classes = 25  # Number of gesture classes
model = create_gesture_model(input_shape, num_classes)
model.summary()

2024-12-29 03:10:37.187282: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_UNSUPPORTED_PTX_VERSION'

2024-12-29 03:10:37.187320: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'

2024-12-29 03:10:37.187335: W tensorflow/core/framework/op_kernel.cc:1829] INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'


InternalError: {{function_node __wrapped__FloorMod_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:FloorMod] name: 

In [ ]:
tenso